In [70]:
import pandas as pd

In [71]:
# Add your file paths
file_paths = [
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Albuquerque_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Anaheim_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Arlington_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Atlanta_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/AuroraCO_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Austin_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Baltimore_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Boston_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Buffalo_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/CapeCoral_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/ColoradoSprings_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Columbus_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Dallas_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/DesMoines_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Detroit_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Durham_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Fresno_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/GardenGrove_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/GrandRapids_Final_2022-06-18.csv",
      "https://raw.githubusercontent.com/imblackline/Sigma/main/data/Greensboro_Final_2022-06-18.csv",
    ]

In [72]:
# Read each CSV file into a DataFrame
dfs = [pd.read_csv(file_path,dtype={'column_name': 'str'}, low_memory=False) for file_path in file_paths]

In [74]:
len(dfs)

20

In [75]:
# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

In [76]:
len(merged_df)

846135

In [77]:
 # Group by 'state', 'city', and 'scientific_name' and count occurrences
grouped_df = merged_df.groupby(['state', 'city', 'scientific_name']).size().reset_index(name='count')
grouped_df

,state,city,scientific_name,count
0,California,Anaheim,Acacia,34
1,California,Anaheim,Acacia baileyana,5
2,California,Anaheim,Acacia cognata,1
3,California,Anaheim,Acacia decurrens,3
4,California,Anaheim,Acacia longifolia,1
...,...,...,...,...
4902,Texas,Dallas,Ulmus crassifolia,844
4903,Texas,Dallas,Ulmus parvifolia,290
4904,Texas,Dallas,Ulmus pumila,45
4905,Texas,Dallas,Vitex agnus-castus,10


In [78]:
# Get the top 10 scientific names
top_scientific_names = grouped_df.groupby('scientific_name')['count'].sum().nlargest(10).index.tolist()
top_scientific_names

['Acer platanoides',
 'Acer rubrum',
 'Gleditsia triacanthos',
 'Tilia cordata',
 'Fraxinus pennsylvanica',
 'Pyrus calleryana',
 'Acer saccharinum',
 'Acer saccharum',
 'Malus',
 'Platanus acerifolia']

In [79]:
# Filter the grouped DataFrame to keep only rows with top 10 scientific names
filtered_df = grouped_df[grouped_df['scientific_name'].isin(top_scientific_names)]
# filtered_df = filtered_df.sort_values(by='count', ascending=False)
filtered_df

,state,city,scientific_name,count
12,California,Anaheim,Acer rubrum,15
13,California,Anaheim,Acer saccharinum,44
136,California,Anaheim,Gleditsia triacanthos,270
243,California,Anaheim,Platanus acerifolia,1264
266,California,Anaheim,Pyrus calleryana,320
...,...,...,...,...
4861,Texas,Dallas,Acer saccharinum,27
4862,Texas,Dallas,Acer saccharum,3
4870,Texas,Dallas,Fraxinus pennsylvanica,17
4872,Texas,Dallas,Gleditsia triacanthos,19


In [ ]:
# Create a dictionary to store the state-city relationships
state_city_dict = {}
for state, city, count in filtered_df.groupby(['state', 'city'])['count'].sum().reset_index().values:
    state_city_dict[(state, city)] = count

state_city_dict

In [84]:
# Create a dictionary to store the city-scientific_name relationships
city_scientific_name_dict = {}
for city, scientific_name, count in filtered_df.groupby(['city', 'scientific_name'])['count'].sum().reset_index().values:
    city_scientific_name_dict[(city, scientific_name)] = count



In [85]:
# Create the final list in the specified format
result_list = [
    {"source": state, "target": city, "value": str(count)}
    for (state, city), count in state_city_dict.items()
] + [
    {"source": city, "target": scientific_name, "value": str(count)}
    for (city, scientific_name), count in city_scientific_name_dict.items()
]

In [ ]:
result_list

In [90]:
import csv
# Replace the path with the desired file path and name
csv_file_path = 'filtered_data.csv'

# Export result_list as CSV
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['source', 'target', 'value']
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header
    csv_writer.writeheader()

    # Write the data
    csv_writer.writerows(result_list)